In [182]:
import pandas as pd

In [183]:
data_raw = pd.read_csv("COVID19MEXICO.csv")

In [184]:
data_raw.columns

Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'INDIGENA',
       'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'TOMA_MUESTRA_LAB', 'RESULTADO_LAB',
       'TOMA_MUESTRA_ANTIGENO', 'RESULTADO_ANTIGENO', 'CLASIFICACION_FINAL',
       'MIGRANTE', 'PAIS_NACIONALIDAD', 'PAIS_ORIGEN', 'UCI'],
      dtype='object')

In [185]:
data_raw[['FECHA_ACTUALIZACION','FECHA_SINTOMAS','FECHA_SINTOMAS']].head()

,FECHA_ACTUALIZACION,FECHA_SINTOMAS,FECHA_SINTOMAS
0,2024-04-16,2023-06-09,2023-06-09
1,2024-04-16,2023-01-17,2023-01-17
2,2024-04-16,2023-05-26,2023-05-26
3,2024-04-16,2023-01-14,2023-01-14
4,2024-04-16,2023-01-16,2023-01-16


In [186]:
data = data_raw.loc[:,['FECHA_ACTUALIZACION', 'FECHA_SINTOMAS', 'ENTIDAD_NAC', 'CLASIFICACION_FINAL','ENTIDAD_RES']]

In [187]:
estados_id = {
    1: "Aguascalientes",
    2: "Baja California",
    3: "Baja California Sur",
    4: "Campeche",
    5: "Chiapas",
    6: "Chihuahua",
    7: "Ciudad de México",
    8: "Coahuila",
    9: "Colima",
    10: "Durango",
    11: "Estado de México",
    12: "Guanajuato",
    13: "Guerrero",
    14: "Hidalgo",
    15: "Jalisco",
    16: "Michoacán",
    17: "Morelos",
    18: "Nayarit",
    19: "Nuevo León",
    20: "Oaxaca",
    21: "Puebla",
    22: "Querétaro",
    23: "Quintana Roo",
    24: "San Luis Potosí",
    25: "Sinaloa",
    26: "Sonora",
    27: "Tabasco",
    28: "Tamaulipas",
    29: "Tlaxcala",
    30: "Veracruz",
    31: "Yucatán",
    32: "Zacatecas"
}

data['ENTIDAD_NAC'] = data['ENTIDAD_NAC'].replace(estados_id)
data['ENTIDAD_RES'] = data['ENTIDAD_RES'].replace(estados_id)

In [188]:
data.head()

,FECHA_ACTUALIZACION,FECHA_SINTOMAS,ENTIDAD_NAC,CLASIFICACION_FINAL,ENTIDAD_RES
0,2024-04-16,2023-06-09,Jalisco,7,Baja California Sur
1,2024-04-16,2023-01-17,Sinaloa,6,Nayarit
2,2024-04-16,2023-05-26,Hidalgo,6,Hidalgo
3,2024-04-16,2023-01-14,Baja California Sur,7,Baja California Sur
4,2024-04-16,2023-01-16,Ciudad de México,6,Ciudad de México


In [189]:
from geopy.geocoders import Bing
import pandas as pd

def obtener_coordenadas_estados(tabla_estados):
    coords = []
    geolocator = Bing(api_key='Ago97Rd3jwxWgFxgUWwgUrZ7S21qlE8ok8UvjoEnaXP2nl2Uh8NHqokudoSjLJzI')  
    for estado in tabla_estados['nombre_estado']:
        query = f"{estado}, México"

        location = geolocator.geocode(query)

        if location:
            latitude = location.latitude
            longitude = location.longitude
            coords.append((estado, latitude, longitude))
        else:
            # Si no se encuentra la localización, se retorna None para ambos
            coords.append((estado, None, None))

    # Crear un DataFrame para el retorno de datos
    return pd.DataFrame(coords, columns=['nombre_estado', 'latitud', 'longitud'])

# Suponiendo que tienes un DataFrame con los nombres de los estados
nombres_estados = pd.DataFrame({
    'nombre_estado': ["Aguascalientes", "Baja California", "Baja California Sur", "Campeche", "Chiapas", "Chihuahua", "Ciudad de México", "Coahuila", "Colima", "Durango", "Guanajuato", "Guerrero", "Hidalgo", "Jalisco", "Estado de México", "Michoacán", "Morelos", "Nayarit", "Nuevo León", "Oaxaca", "Puebla", "Querétaro", "Quintana Roo", "San Luis Potosí", "Sinaloa", "Sonora", "Tabasco", "Tamaulipas", "Tlaxcala", "Veracruz", "Yucatán", "Zacatecas"]
})

# Llamada a la función
coordenadas_estados = obtener_coordenadas_estados(nombres_estados)
coordenadas_estados = pd.DataFrame(coordenadas_estados)


In [190]:
# Duplicar y renombrar para ENTIDAD_NAC
coordenadas_nac = coordenadas_estados.rename(columns={'nombre_estado': 'ENTIDAD_NAC', 'latitud': 'latitud_nac', 'longitud': 'longitud_nac'})

# Duplicar y renombrar para ENTIDAD_RES
coordenadas_res = coordenadas_estados.rename(columns={'nombre_estado': 'ENTIDAD_RES', 'latitud': 'latitud_res', 'longitud': 'longitud_res'})

In [191]:
# Merge para ENTIDAD_NAC
data = pd.merge(data, coordenadas_nac[['ENTIDAD_NAC', 'latitud_nac', 'longitud_nac']], on='ENTIDAD_NAC', how='left')

# Merge para ENTIDAD_RES
data = pd.merge(data, coordenadas_res[['ENTIDAD_RES', 'latitud_res', 'longitud_res']], on='ENTIDAD_RES', how='left')


In [192]:
data.head()

,FECHA_ACTUALIZACION,FECHA_SINTOMAS,ENTIDAD_NAC,CLASIFICACION_FINAL,ENTIDAD_RES,latitud_nac,longitud_nac,latitud_res,longitud_res
0,2024-04-16,2023-06-09,Jalisco,7,Baja California Sur,20.580795,-103.613129,25.938501,-112.061226
1,2024-04-16,2023-01-17,Sinaloa,6,Nayarit,24.997808,-107.477928,21.804178,-104.840919
2,2024-04-16,2023-05-26,Hidalgo,6,Hidalgo,20.479559,-98.887062,20.479559,-98.887062
3,2024-04-16,2023-01-14,Baja California Sur,7,Baja California Sur,25.938501,-112.061226,25.938501,-112.061226
4,2024-04-16,2023-01-16,Ciudad de México,6,Ciudad de México,19.432678,-99.134209,19.432678,-99.134209


In [193]:
data = data.dropna(subset=['latitud_nac', 'longitud_nac', 'latitud_res', 'longitud_res'])

In [194]:
data = data.loc[:,['FECHA_ACTUALIZACION', 'FECHA_SINTOMAS', 'ENTIDAD_NAC','latitud_nac','longitud_nac','ENTIDAD_RES','latitud_res','longitud_res','CLASIFICACION_FINAL']]

In [195]:

data_final = data[data['CLASIFICACION_FINAL'].isin([1, 2, 3])]


In [196]:
data_final = data_final.reset_index(drop=True)

In [197]:
data_final

,FECHA_ACTUALIZACION,FECHA_SINTOMAS,ENTIDAD_NAC,latitud_nac,longitud_nac,ENTIDAD_RES,latitud_res,longitud_res,CLASIFICACION_FINAL
0,2024-04-16,2023-01-30,Colima,19.242922,-103.728119,Colima,19.242922,-103.728119,3
1,2024-04-16,2023-02-14,Baja California Sur,25.938501,-112.061226,Baja California Sur,25.938501,-112.061226,3
2,2024-04-16,2023-01-08,Jalisco,20.580795,-103.613129,Colima,19.242922,-103.728119,3
3,2024-04-16,2023-01-02,Guerrero,17.668085,-99.921646,Colima,19.242922,-103.728119,3
4,2024-04-16,2023-05-08,Colima,19.242922,-103.728119,Colima,19.242922,-103.728119,3
...,...,...,...,...,...,...,...,...,...
433185,2024-04-16,2023-08-27,Colima,19.242922,-103.728119,Jalisco,20.580795,-103.613129,3
433186,2024-04-16,2023-08-28,Colima,19.242922,-103.728119,Jalisco,20.580795,-103.613129,3
433187,2024-04-16,2023-08-28,Jalisco,20.580795,-103.613129,Jalisco,20.580795,-103.613129,3
433188,2024-04-16,2023-08-29,Colima,19.242922,-103.728119,Colima,19.242922,-103.728119,3


In [198]:
# Exportar el DataFrame a CSV con delimitador de tabulaciones y manejo específico de valores nulos
data_final.to_csv('COVID19.csv', index=False, encoding='utf-8')